<a href="https://colab.research.google.com/github/BigTMiami/AdaptOrDie/blob/main/AAa_REVIEW_active_training_adapters_adp_seq_bn_P_micro_seq_bn_C_micro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Summary
This creates one adapter for pre-training and another for classifcation.  

* The pre-training adapter is not trained during the classification, only the classifcation adapter.
* Naming Convention - (pretraining adapter type)_P_(pretraining data size)_(classification adapter type)_C_(classification_data_size)
* This uses "micro" sizes - just to demonstrate the script works

# TODO
* Set eval steps in training
* Set logging steps more appropriately





# Setup

In [1]:
from time import time
start_time = time()

In [2]:
experiment_tag = "AAa"

# Pretraining Dataset Settings
pretrain_dataset_name = "BigTMiami/amazon_split_25M_reviews_20_percent_condensed"
pretrain_percentage_int = 20
pretrain_percentage = f"{pretrain_percentage_int}"
pretrain_train_split = f"train[:{pretrain_percentage}%]"
pretrain_eval_split = "validation"
# The dataset I am using is only 20%, so the final is 20% of the pretrain_percentage_int
pretrain_percentage_final = int(0.2 * pretrain_percentage_int)

# MICRO Dataset Settings - comment out for regular run
pretrain_dataset_name = "BigTMiami/amazon_25M_10_000_condensed"
pretrain_train_split = f"train"
pretrain_eval_split = "validation"
pretrain_percentage_final = "micro"

# Pretraining Adapter settings
pretraining_adapter_type = "seq_bn"
pretraining_tag = f"{pretraining_adapter_type}_P_{pretrain_percentage_final}"
pretrained_adapter_hub_name = f"{experiment_tag}_adp_{pretraining_tag}"
pretrained_adapter_name = f"{pretrained_adapter_hub_name}_adapter"
print(f"Pretrained Adapter Hub Name: {pretrained_adapter_hub_name}")

# Classification Dataset Settings
classification_dataset_name = "BigTMiami/amazon_helpfulness"
classification_train_percentage = "1"
classification_train_split = f"train[:{classification_train_percentage}%]"
classification_eval_split = "dev"
classfication_percentage_final = "micro"

# Classification Adapter settings
classification_adapter_type = "seq_bn"
classifier_tag = f"{classification_adapter_type}_C_{classfication_percentage_final}"

# Classifier - no pretraining
classification_adapter_hub_name = f"{experiment_tag}_adp_{classifier_tag}"
classification_adapter_name = f"{classification_adapter_hub_name}_adapter"
print(f"Classifer Not Pretrained Adapter Hub Name: {classification_adapter_hub_name}")

# Classifier with Pretrained
classification_adapter_after_pretrained_hub_name = f"{experiment_tag}_adp_{pretraining_tag}_{classifier_tag}"
classification_adapter_after_pretrained_name = f"{classification_adapter_after_pretrained_hub_name}_adapter"
print(f"Classifer after Pretrained Adapter Hub Name: {classification_adapter_after_pretrained_hub_name}")


Pretrained Adapter Hub Name: AAa_adp_seq_bn_P_micro
Classifer Not Pretrained Adapter Hub Name: AAa_adp_seq_bn_C_micro
Classifer after Pretrained Adapter Hub Name: AAa_adp_seq_bn_P_micro_seq_bn_C_micro


In [3]:
# from google.colab import drive
# drive.mount("/content/drive")
# %cd '/content/drive/MyDrive/AdaptOrDie'

In [4]:
!pip install datasets
!pip install transformers[torch]
!pip install adapters
!pip install scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.4/297.4 kB 6.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x8

In [5]:
!pip install pynvml

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 2.2 MB/s eta 0:00:00


In [6]:
from pynvml import *

def print_gpu_utilization():
    nvmlInit()
    handle = nvmlDeviceGetHandleByIndex(0)
    info = nvmlDeviceGetMemoryInfo(handle)
    gpu_used = info.used//1024**2
    print(f"GPU {gpu_used} MB")

print_gpu_utilization()

GPU 448 MB


In [7]:
from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)

    # Calculate accuracy
    accuracy = accuracy_score(labels, preds)

   # Calculate precision, recall, and F1-score
    f1 = f1_score(labels, preds, average='macro')

    return {
        'accuracy': accuracy,
        'f1_macro': f1
    }

# Pretraining

In [ ]:
from datasets import load_dataset

pretrain_train_dataset, pretrain_eval_dataset = load_dataset(pretrain_dataset_name, split=[pretrain_train_split,pretrain_eval_split])

print(pretrain_train_dataset)
print(pretrain_eval_dataset)

In [ ]:
from transformers import RobertaConfig
from adapters import AutoAdapterModel

config = RobertaConfig.from_pretrained("roberta-base")
pretrain_model = AutoAdapterModel.from_pretrained(
    "roberta-base",
    config=config,
)

In [ ]:
# THIS MUST BE SET TO MATCH  pretraining_adapter_type
from adapters import SeqBnConfig

pretraining_adapter_config = SeqBnConfig()

In [ ]:
print(pretraining_adapter_config.__class__.__name__)
print(pretraining_adapter_type)

In [ ]:
# Add a new adapter
pretrain_model.add_adapter(pretrained_adapter_name, config=pretraining_adapter_config)

# Add head for masked language modeling
pretrain_model.add_masked_lm_head(pretrained_adapter_name)

# Set the adapter to be used for training
pretrain_model.train_adapter(pretrained_adapter_name)

print(f"Pretrained Adapter Name: {pretrained_adapter_name}")
print(pretrain_model.active_adapters)

In [10]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("roberta-base")
print_gpu_utilization()

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

GPU 448 MB


In [ ]:
from transformers import DataCollatorForLanguageModeling

pretraining_data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm_probability=0.15
)

In [ ]:
from transformers import TrainingArguments

pretraining_args = TrainingArguments(
    output_dir=pretrained_adapter_hub_name,
    learning_rate=1e-4,
    num_train_epochs=1,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    gradient_accumulation_steps=11,
    weight_decay=0.01,
    # Logging and eval during training settings
    logging_steps=50,
    evaluation_strategy="steps",
    eval_steps=100,
    overwrite_output_dir=True,
    # The next line is important to ensure the dataset labels are properly passed to the model
    remove_unused_columns=False,
)

In [ ]:
from adapters import AdapterTrainer

pretrain_trainer = AdapterTrainer(
    model=pretrain_model,
    args=pretraining_args,
    train_dataset=pretrain_train_dataset,
    eval_dataset=pretrain_eval_dataset,
    data_collator=pretraining_data_collator,
)

In [ ]:
before_pretrain_eval_results = pretrain_trainer.evaluate()
before_pretrain_eval_results_loss = before_pretrain_eval_results["eval_loss"]
print(f"Before pretraining loss: {before_pretrain_eval_results_loss:.4f}")
print(before_pretrain_eval_results )

In [ ]:
pretrain_results = pretrain_trainer.train()
print(pretrain_results)

In [ ]:
after_pretrain_eval_results = pretrain_trainer.evaluate()
after_pretrain_eval_results_loss = after_pretrain_eval_results["eval_loss"]
print(f"After pretraining loss: {after_pretrain_eval_results_loss:.4f}")
print(after_pretrain_eval_results)

In [ ]:
pretrain_model.push_adapter_to_hub(
    pretrained_adapter_hub_name,
    pretrained_adapter_name,
    datasets_tag=pretrain_dataset_name
)

In [ ]:
import gc

print_gpu_utilization()

del pretraining_data_collator
del pretrain_train_dataset
del pretrain_eval_dataset
del pretrain_model
del pretrain_trainer
gc.collect()

print_gpu_utilization()

# Classification

In [8]:
from datasets import load_dataset

classification_train_dataset, classification_eval_dataset= load_dataset(classification_dataset_name, split=[classification_train_split,classification_eval_split])

print(classification_train_dataset)
print(classification_eval_dataset)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/repocard.py:105: UserWarning: Repo card metadata block was not found. Setting CardData to empty.
  warnings.warn("Repo card metadata block was not found. Setting CardData to empty.")


Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating dev split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['label', 'input_ids', 'attention_mask'],
    num_rows: 1153
})
Dataset({
    features: ['label', 'input_ids', 'attention_mask'],
    num_rows: 5000
})


In [ ]:
from transformers import RobertaConfig
from adapters import AutoAdapterModel

config = RobertaConfig.from_pretrained("roberta-base")
classficiation_no_pretrain_model = AutoAdapterModel.from_pretrained(
    "roberta-base",
    config=config,
)

In [23]:
# THIS MUST BE SET TO MATCH  classification_adapter_type
from adapters import SeqBnConfig

classification_adapter_config = SeqBnConfig()

In [ ]:
print(classification_adapter_config.__class__.__name__)
print(classification_adapter_type)

In [ ]:
# Add a new adapter
classficiation_no_pretrain_model.add_adapter(classification_adapter_name, config=classification_adapter_config)

# Add head for classification modeling
classficiation_no_pretrain_model.add_classification_head(
    classification_adapter_name,
    num_labels=2,
    id2label={ 0: "unhelpful", 1: "helpful"})

# Set the adapter to be used for training
classficiation_no_pretrain_model.train_adapter(classification_adapter_name)

print(f"Classification Adapter Name: {classification_adapter_name}")
print(classficiation_no_pretrain_model.active_adapters)

In [11]:
from transformers import DataCollatorWithPadding

classification_data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
from transformers import TrainingArguments

classification_training_args = TrainingArguments(
    output_dir=classification_adapter_hub_name,
    learning_rate=1e-4,
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    # Logging and eval during training settings
    logging_steps=500,
    evaluation_strategy="steps",
    eval_steps=1000,
    overwrite_output_dir=True,
    # The next line is important to ensure the dataset labels are properly passed to the model
    remove_unused_columns=False,
)

In [ ]:
from adapters import AdapterTrainer

classification_trainer = AdapterTrainer(
    model=classficiation_no_pretrain_model,
    args=classification_training_args,
    train_dataset=classification_train_dataset,
    eval_dataset=classification_eval_dataset,
    data_collator=classification_data_collator,
    compute_metrics=compute_metrics,
)

In [ ]:
classification_training_results = classification_trainer.train()
print(classification_training_results)

In [ ]:
classification_eval_results = classification_trainer.evaluate()
classification_eval_loss = classification_eval_results["eval_loss"]
classification_eval_f1 = classification_eval_results["eval_f1_macro"]
print(f"Classification no Pretraining Eval F1:{100.0 * classification_eval_f1:.2f}")
print(f"Classification no Pretraining Eval Loss: {classification_eval_loss:.2f}")
print(classification_eval_results)

In [ ]:
classficiation_no_pretrain_model.push_adapter_to_hub(
    classification_adapter_hub_name,
    classification_adapter_name,
    datasets_tag=classification_dataset_name
)
print(f"Pushed {classification_adapter_hub_name}")

In [ ]:
import gc

print_gpu_utilization()

del classficiation_no_pretrain_model
del classification_trainer
gc.collect()

print_gpu_utilization()

# Classification from Pre-training

In [12]:
from transformers import RobertaConfig
from adapters import AutoAdapterModel

config = RobertaConfig.from_pretrained("roberta-base")
classficiation_from_pretrain_model = AutoAdapterModel.from_pretrained(
    "roberta-base",
    config=config,
)

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaAdapterModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight', 'heads.default.3.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
print(pretrained_adapter_hub_name)
BigTMiami/AA_adp_seq_bn_P_micro

AAa_adp_seq_bn_P_micro


In [16]:
# Load pre-trained adapter
load_name = f"BigTMiami/AA_adp_seq_bn_P_micro"

# Load Pretrained adapter without head
loaded_adapter_name = classficiation_from_pretrain_model.load_adapter(load_name, with_head=False)

#Set active below after training is set
classficiation_from_pretrain_model.set_active_adapters(loaded_adapter_name)

print(f"Loaded Pretrain Adapter Name: {load_name}")

Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

README.md:   0%|          | 0.00/1.03k [00:00<?, ?B/s]

adapter_config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

.gitattributes:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

head_config.json:   0%|          | 0.00/431 [00:00<?, ?B/s]

pytorch_adapter.bin:   0%|          | 0.00/3.60M [00:00<?, ?B/s]

pytorch_model_head.bin:   0%|          | 0.00/157M [00:00<?, ?B/s]

Loaded Pretrain Adapter Name: BigTMiami/AA_adp_seq_bn_P_micro
<bound method ModelAdaptersMixin.adapter_summary of RobertaAdapterModel(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttentionWithAdapters(
              (query): LoRALinear(
                in_features=768, out_features=768, bias=True
                (loras): ModuleDict()
              )
              (key): LoRALinear(
                in_features=768, out_features=768, bias=True
                (loras): ModuleDict()
              )
              

In [30]:
summary = classficiation_from_pretrain_model.adapter_summary()
print(summary)

Name                     Architecture         #Param      %Param  Active   Train
--------------------------------------------------------------------------------
AA_adp_seq_bn_P_micro_adapterbottleneck          894,528       0.718       0       0
AAa_adp_seq_bn_P_micro_seq_bn_C_micro_adapterbottleneck          894,528       0.718       1       1
--------------------------------------------------------------------------------
Full model                               124,645,632     100.000               0


In [19]:
print(classficiation_from_pretrain_model.active_adapters)

Stack[AA_adp_seq_bn_P_micro_adapter]


In [ ]:
# classification_adapter_config set above for no classification

In [24]:
# Add a new adapter
classficiation_from_pretrain_model.add_adapter(classification_adapter_after_pretrained_name, config=classification_adapter_config)

# Add head for classification modeling
classficiation_from_pretrain_model.add_classification_head(
    classification_adapter_after_pretrained_name,
    num_labels=2,
    id2label={ 0: "unhelpful", 1: "helpful"})

# Set the adapter to be used for training
classficiation_from_pretrain_model.train_adapter(classification_adapter_after_pretrained_name)

print(f"Classification after Pretrain Adapter Name: {classification_adapter_after_pretrained_name}")


Classification after Pretrain Adapter Name: AAa_adp_seq_bn_P_micro_seq_bn_C_micro_adapter


In [33]:
classficiation_from_pretrain_model.set_active_adapters([loaded_adapter_name, classification_adapter_after_pretrained_name])
print(classficiation_from_pretrain_model.active_adapters)
summary = classficiation_from_pretrain_model.adapter_summary()
print(summary)

Stack[AA_adp_seq_bn_P_micro_adapter, AAa_adp_seq_bn_P_micro_seq_bn_C_micro_adapter]
Name                     Architecture         #Param      %Param  Active   Train
--------------------------------------------------------------------------------
AA_adp_seq_bn_P_micro_adapterbottleneck          894,528       0.718       1       0
AAa_adp_seq_bn_P_micro_seq_bn_C_micro_adapterbottleneck          894,528       0.718       1       1
--------------------------------------------------------------------------------
Full model                               124,645,632     100.000               0


In [34]:
from transformers import TrainingArguments

classification_from_pretrained_training_args = TrainingArguments(
    output_dir=classification_adapter_after_pretrained_hub_name,
    learning_rate=1e-4,
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    # Logging and eval during training settings
    logging_steps=500,
    evaluation_strategy="steps",
    eval_steps=1000,
    overwrite_output_dir=True,
    # The next line is important to ensure the dataset labels are properly passed to the model
    remove_unused_columns=False,
)

In [35]:
from adapters import AdapterTrainer

classification_from_pretrained_trainer = AdapterTrainer(
    model=classficiation_from_pretrain_model,
    args=classification_from_pretrained_training_args,
    train_dataset=classification_train_dataset,
    eval_dataset=classification_eval_dataset,
    data_collator=classification_data_collator,
    compute_metrics=compute_metrics,
)
print_gpu_utilization()

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False)
  warnings.warn(


GPU 1405 MB


In [36]:
classification_from_pretrained_training_results = classification_from_pretrained_trainer.train()
print(classification_from_pretrained_training_results)
print_gpu_utilization()

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss


TrainOutput(global_step=219, training_loss=0.42995689775301443, metrics={'train_runtime': 59.2115, 'train_samples_per_second': 58.418, 'train_steps_per_second': 3.699, 'total_flos': 878281590561840.0, 'train_loss': 0.42995689775301443, 'epoch': 3.0})
GPU 10289 MB


In [ ]:
classification_from_pretrained_eval_results = classification_from_pretrained_trainer.evaluate()
classification_from_pretrained_eval_loss = classification_from_pretrained_eval_results["eval_loss"]
classification_from_pretrained_eval_f1 = classification_from_pretrained_eval_results["eval_f1_macro"]
print(f"Classification with Pretraining Eval F1:{100.0 * classification_from_pretrained_eval_f1:.2f}")
print(f"Classification with Pretraining Eval Loss: {classification_from_pretrained_eval_loss:.2f}")
print(classification_from_pretrained_eval_results)
print_gpu_utilization()

In [ ]:
classficiation_from_pretrain_model.push_adapter_to_hub(
    classification_adapter_after_pretrained_hub_name,
    classification_adapter_after_pretrained_name,
    datasets_tag=classification_dataset_name
)
print(f"Pushed {classification_adapter_after_pretrained_hub_name}")

In [ ]:
print_gpu_utilization()

In [ ]:
end_time = time()
total_time = end_time - start_time

In [ ]:
print("Summary")
print("----------------")
print(f"Pretraining Adapter Type:{pretraining_adapter_type}")
print(f"Pretrain Overall Train Percentage:{pretrain_percentage_final}")
print("")
print(f"Classification Adapter Type:{classification_adapter_type}")
print(f"Classification Train Percentage:{classfication_percentage_final}")
print("")
print("Running Time")
print("----------------")
print(f"Total Time: {total_time:.0f} seconds OR {total_time / 60.0 :.0f} minutes OR {total_time / 3600.0 :.1f} hours ")
print("")
print("Pre-training Loss Change - should be high because adapter is completely random")
print("----------------")
print(f"Before pretraining loss: {before_pretrain_eval_results_loss:.4f}")
print(f"After pretraining loss: {after_pretrain_eval_results_loss:.4f}")
print("")
print("Classification Loss Change - should not be that different because both are starting from a blank classifier")
print("----------------")
print(f"Classification no Pretraining Eval Loss: {classification_eval_loss:.4f}")
print(f"Classification with Pretraining Eval Loss: {classification_from_pretrained_eval_loss:.4f}")
print("")
print("Classification F1 Change - hopefully we get a few points improvement here!")
print("----------------")
print(f"Classification no Pretraining Eval F1:{100.0 * classification_eval_f1:.2f}")
print(f"Classification with Pretraining Eval F1:{100.0 * classification_from_pretrained_eval_f1:.2f}")

print("")
pretraining_f1_improvement = (100.0 * (  classification_from_pretrained_eval_f1 - classification_eval_f1))
print(f"Pretraining F1 Improvement: {pretraining_f1_improvement:.2f}")

In [ ]:
print("Disconnecting Session")
from google.colab import runtime
runtime.unassign()